# EEE586 Assignment 1 - Burak Taşdemir 22003996

## Includes

In [3]:
import string
import collections
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
import seaborn as sns
import os
import pathlib
import pickle
import math

In [1]:
!export PATH=/Library/TeX/texbin:$PATH

In [2]:
jupyter nbconvert EEE586_Assignment_1.ipynb --to pdf

SyntaxError: invalid syntax (3125126678.py, line 1)

## Functions

In [4]:
def clean_begin_end_text(text):
    start_fence = "start of the project gutenberg"
    end_fence = "end of the project gutenberg"
    text = text.lower()
    start_pos = text.find(start_fence) + len(start_fence) + 1
    end_pos = text.find(end_fence)

    # Check that the fences are at reasonable positions within the text.
    assert 0.000001 < start_pos / len(text) <= 0.1
    #print(end_pos)
    #print(len(text))
    assert 0.9 < end_pos / len(text) <= 1.0

    return text[start_pos:end_pos]

In [5]:
def stopword_removal(text):
    #default stopwrods vs stacy stopwords
    stop=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    stop2=['those', 'on', 'own', '’ve', 'yourselves', 'around', 'between', 'four', 'been', 'alone', 'off', 'am', 'then', 'other', 'can', 'regarding', 'hereafter', 'front', 'too', 'used', 'wherein', '‘ll', 'doing', 'everything', 'up', 'onto', 'never', 'either', 'how', 'before', 'anyway', 'since', 'through', 'amount', 'now', 'he', 'was', 'have', 'into', 'because', 'not', 'therefore', 'they', 'n’t', 'even', 'whom', 'it', 'see', 'somewhere', 'thereupon', 'nothing', 'whereas', 'much', 'whenever', 'seem', 'until', 'whereby', 'at', 'also', 'some', 'last', 'than', 'get', 'already', 'our', 'once', 'will', 'noone', "'m", 'that', 'what', 'thus', 'no', 'myself', 'out', 'next', 'whatever', 'although', 'though', 'which', 'would', 'therein', 'nor', 'somehow', 'whereupon', 'besides', 'whoever', 'ourselves', 'few', 'did', 'without', 'third', 'anything', 'twelve', 'against', 'while', 'twenty', 'if', 'however', 'herself', 'when', 'may', 'ours', 'six', 'done', 'seems', 'else', 'call', 'perhaps', 'had', 'nevertheless', 'where', 'otherwise', 'still', 'within', 'its', 'for', 'together', 'elsewhere', 'throughout', 'of', 'others', 'show', '’s', 'anywhere', 'anyhow', 'as', 'are', 'the', 'hence', 'something', 'hereby', 'nowhere', 'latterly', 'say', 'does', 'neither', 'his', 'go', 'forty', 'put', 'their', 'by', 'namely', 'could', 'five', 'unless', 'itself', 'is', 'nine', 'whereafter', 'down', 'bottom', 'thereby', 'such', 'both', 'she', 'become', 'whole', 'who', 'yourself', 'every', 'thru', 'except', 'very', 'several', 'among', 'being', 'be', 'mine', 'further', 'n‘t', 'here', 'during', 'why', 'with', 'just', "'s", 'becomes', '’ll', 'about', 'a', 'using', 'seeming', "'d", "'ll", "'re", 'due', 'wherever', 'beforehand', 'fifty', 'becoming', 'might', 'amongst', 'my', 'empty', 'thence', 'thereafter', 'almost', 'least', 'someone', 'often', 'from', 'keep', 'him', 'or', '‘m', 'top', 'her', 'nobody', 'sometime', 'across', '‘s', '’re', 'hundred', 'only', 'via', 'name', 'eight', 'three', 'back', 'to', 'all', 'became', 'move', 'me', 'we', 'formerly', 'so', 'i', 'whence', 'under', 'always', 'himself', 'in', 'herein', 'more', 'after', 'themselves', 'you', 'above', 'sixty', 'them', 'your', 'made', 'indeed', 'most', 'everywhere', 'fifteen', 'but', 'must', 'along', 'beside', 'hers', 'side', 'former', 'anyone', 'full', 'has', 'yours', 'whose', 'behind', 'please', 'ten', 'seemed', 'sometimes', 'should', 'over', 'take', 'each', 'same', 'rather', 'really', 'latter', 'and', 'ca', 'hereupon', 'part', 'per', 'eleven', 'ever', '‘re', 'enough', "n't", 'again', '‘d', 'us', 'yet', 'moreover', 'mostly', 'one', 'meanwhile', 'whither', 'there', 'toward', '’m', "'ve", '’d', 'give', 'do', 'an', 'quite', 'these', 'everyone', 'towards', 'this', 'cannot', 'afterwards', 'beyond', 'make', 'were', 'whether', 'well', 'another', 'below', 'first', 'upon', 'any', 'none', 'many', 'serious', 'various', 're', 'two', 'less', '‘ve']
    nonstop_words = [i for i in text if i not in stop2]
    return nonstop_words

In [6]:
def count_words(text, clean_text=False):
    #text = f.read().encode().decode(errors='replace')
    #text = open(p, 'rb').read().decode('ISO-8859-1')
    if clean_text:
        text = clean_begin_end_text(text)

    #chunks = text.split()
    #nonstopchunks = stopword_removal(chunks):
    npunc = [word.strip(string.punctuation) for word in chunks]
    #npunc_nonstop = [nonstop_word.strip(string.punctuation) for nonstop_word in nonstop_chunks]
    word_list = [word.lower() for word in npunc if word]
    #nonstop_word_list = [nonstop_word.lower() for word in npunc_nonstop if nonstop_word]
    #nonstop_word_counts = collections.Counter(nonstop_word_list)
    word_counts = collections.Counter(word_list)
    
    return dict(word_counts)

In [7]:
def plot_zipf(source,number,word_counts,regularFlag):
    if source=='cd':
        source='Charles Dickens'
    elif source=='mary':
        source='Mary Shelley'
    elif source=='fitz':
        source='F. Scott Fitzerald'
    values = [k for k,l in sorted([(j,i) for i,j in word_counts.items()], reverse=True)]
    log_values = [math.log(k) for k,l in sorted([(j,i) for i,j in word_counts.items()], reverse = True)]
    x = [i+1 for i in range(len(values))]
    log_x=[math.log(i) for i in x]
    #print(log_x)
    fig1 = plt.figure(figsize= (16,10))

    
    plt.plot(x,values, 'o',label = source+" books "+" "+regularFlag)
    plt.xlabel('Rank')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
    fig2 = plt.figure(figsize= (16,10))
    
    plt.plot(log_x, log_values, 'o',label = source+" books "+" "+regularFlag )
    plt.xlabel('Log(Rank)')
    plt.ylabel('Log(Frequency)')
    plt.legend()
    plt.show()

    #plt.title("Zipf Law Log-Log plot for "+source+" "+regularFlag)

In [8]:
def plot_zipf_different(source,word_counts,regularFlag):
    if source=='cd':
        source='Charles Dickens'
    elif source=='mary':
        source='Mary Shelley'
    elif source=='fitz':
        source='F. Scott Fitzerald'
    fig = plt.figure(figsize= (16,10))
    for i in range(3):
        values = [k for k,l in sorted([(j,i) for i,j in word_counts[i].items()], reverse=True)]
        log_values = [math.log(k) for k,l in sorted([(j,i) for i,j in word_counts[i].items()], reverse = True)]
        x = [i+1 for i in range(len(values))]
        log_x=[math.log(i) for i in x]
        plt.plot(log_x, log_values, 'o', label = source+" book "+str(i+1)+" "+regularFlag)
    plt.xlabel('Log(Rank)')
    plt.ylabel('Log(Frequency)')
    plt.legend()
    #plt.title("Zipf Law Log-Log plot for "+source+" "+regularFlag)
    plt.show()

    

In [9]:
def count_unique(word_list):
    word_counts = collections.Counter(word_list)
    return [len(word_list),len(word_counts)]

In [10]:
def plot_unique(source,regular_token_size,regular_word_type,nonstop_token_size,nonstop_word_type):
    if source=='cd':
        source='Charles Dickens'
    elif source=='mary':
        source='Mary Shelley'
    elif source=='fitz':
        source='F. Scott Fitzerald'
    regular_log_token_size = [math.log(i) for i in regular_token_size]
    regular_log_word_type = [math.log(i) for i in regular_word_type]
    nonstop_log_token_size = [math.log(i) for i in nonstop_token_size]
    nonstop_log_word_type = [math.log(i) for i in nonstop_word_type]

    fig1 = plt.figure(figsize= (16,10))

    
    plt.plot(regular_token_size,regular_word_type, 'o', label=source+" books"+" "+'with stopwords')
    plt.xlabel('Token Size')
    plt.ylabel('Word Type')
    plt.legend()
    plt.show()
    fig2 = plt.figure(figsize= (16,10))
    plt.plot(regular_log_token_size, regular_log_word_type, 'o', label=source+" books"+" "+'with stopwords' )
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.show()

    fig3 = plt.figure(figsize= (16,10))
    
    plt.plot(nonstop_token_size,nonstop_word_type, 'o', label=source+" books"+" "+'without stopwords')
    plt.xlabel('Token Size')
    plt.ylabel('Word Type')
    plt.legend()
    plt.show()
    fig4 = plt.figure(figsize= (16,10))
    plt.plot(nonstop_log_token_size, nonstop_log_word_type, 'o', label=source+" books"+" "+'without stopwords' )
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.show()

In [11]:
def plot_unique_just_regular(regular_token_size,regular_word_type):
    regular_log_token_size = [math.log(i) for i in regular_token_size]
    regular_log_word_type = [math.log(i) for i in regular_word_type]
    fig1 = plt.figure(figsize= (10,8))

    
    plt.plot(regular_token_size,regular_word_type, 'o', label='random generated corpus')
    plt.xlabel('Token Size')
    plt.ylabel('Word Type')
    plt.legend()
    plt.show()
    fig2 = plt.figure(figsize= (10,8))
    plt.plot(regular_log_token_size, regular_log_word_type, 'o',label='random generated corpus' )
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.show()
    


In [12]:
def plot_unique_different(source, regular_token_size,regular_word_type,nonstop_token_size,nonstop_word_type):
    if source=='cd':
        source='Charles Dickens'
    elif source=='mary':
        source='Mary Shelley'
    elif source=='fitz':
        source='F. Scott Fitzerald'
    elif source=='horror':
        source='Horror Genre'
    elif source=='myth':
        source='Mythology Genre'
    elif source=='crime':
        source='Crime Genre'
    fig1 = plt.figure(figsize= (10,8))    
    for ind in range(3):
        fitted_line=[]
        #regular_log_token_size=[]
        #regular_log_word_type=[]
        regular_log_token_size=[math.log(i) for i in regular_token_size[ind]]
        regular_log_word_type=[math.log(i) for i in regular_word_type[ind]]
        #print(len(regular_log_token_size),len(regular_log_word_type))
        a, b = np.polyfit(regular_log_token_size, regular_log_word_type, 1)
        plt.plot(regular_log_token_size, regular_log_word_type, 'o',label = source+" book"+str(ind+1)+' with stopwords' )
        #print(a,b)
        for ix, num in enumerate(regular_log_token_size):
            fitted_line.append(regular_log_token_size[ix]*a+b)
        #print(regular_log_token_size)
        plt.plot(regular_log_token_size, fitted_line, label = source+" book"+str(ind+1)+' with stopwords'+" slope is: "+str(a))
        
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.title("regular")
    plt.show()
    fig2 = plt.figure(figsize= (10,8))
    for ind_ in range(3):   
        fitted_line=[]
        nonstop_log_token_size=[math.log(i) for i in nonstop_token_size[ind_]]
        nonstop_log_word_type=[math.log(i) for i in nonstop_word_type[ind_]]
        a, b = np.polyfit(nonstop_log_token_size, nonstop_log_word_type, 1)
        for ix, num in enumerate(nonstop_log_token_size):
            fitted_line.append(nonstop_log_token_size[ix]*a+b)
        plt.plot(nonstop_log_token_size, nonstop_log_word_type, 'o',label = source+" book"+str(ind_+1)+' without stopwords' )
        plt.plot(nonstop_log_token_size, fitted_line, label = source+" book"+str(ind+1)+' without stopwords'+" slope is: "+str(a))
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.title("nonstop")
    plt.show()



In [13]:
def plot_all_together( all_token_regular,all_type_regular,all_token_nonstop,all_type_nonstop):
    
    fig1 = plt.figure(figsize= (20,12))    
    for ind in range(3):
        for jnd in range(3):
            fitted_line=[]
            #regular_log_token_size=[]
            #regular_log_word_type=[]
            regular_log_token_size=[math.log(i) for i in all_token_regular[ind][jnd]]
            regular_log_word_type=[math.log(i) for i in all_type_regular[ind][jnd]]
            
            #print(len(regular_log_token_size),len(regular_log_word_type))
            if ind==0:
                plot_color='green'
            if ind==1:
                plot_color='red'
            if ind==2:
                plot_color='blue'

            plt.plot(regular_log_token_size, regular_log_word_type, 'o',label = " book "+str(jnd+1)+" of genre/author "+ str(ind+1)+' with stopwords',color=plot_color )
            a, b = np.polyfit(regular_log_token_size, regular_log_word_type, 1)
            for ix, num in enumerate(regular_log_token_size):
                fitted_line.append(regular_log_token_size[ix]*a+b)
            plt.plot(regular_log_token_size, fitted_line, label = " book "+str(jnd+1)+" of genre/author "+ str(ind+1)+' with stopwords'+" slope is: "+str(a),color=plot_color )
            
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.title("regular")
    plt.show()
    fig2 = plt.figure(figsize= (20,12))
    for ind in range(3):
        for jnd in range(3):
            fitted_line=[]
            nonstop_log_token_size=[math.log(i) for i in all_token_nonstop[ind][jnd]]
            nonstop_log_word_type=[math.log(i) for i in all_type_nonstop[ind][jnd]]
            if ind==0:
                plot_color='green'
            if ind==1:
                plot_color='red'
            if ind==2:
                plot_color='blue'
            plt.plot(nonstop_log_token_size, nonstop_log_word_type, 'o',label = " book "+str(jnd+1)+" of genre/author "+ str(ind+1)+' without stopwords',color=plot_color )
            a, b = np.polyfit(nonstop_log_token_size, nonstop_log_word_type, 1)
            for ix, num in enumerate(nonstop_log_token_size):
                fitted_line.append(nonstop_log_token_size[ix]*a+b)
            plt.plot(nonstop_log_token_size, fitted_line, label = " book "+str(jnd+1)+" of genre/author "+ str(ind+1)+' without stopwords'+" slope is: "+str(a),color=plot_color )
            
    plt.xlabel('Log(Token Size)')
    plt.ylabel('Log(Word Type)')
    plt.legend()
    plt.title("nonstop")
    plt.show()

## Experiments

In [12]:
#advised to be run once
folders=["different_genres","different_authors","merged_genres","merged_authors"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        #print(p)
        read_text = open(p, 'rb').read().decode()
        text = read_text.split()
        npunc = [word.strip(string.punctuation) for word in text]
        word_list = [word.lower() for word in npunc if word]
        
        #PART D
        nonstop_word_list=stopword_removal(word_list)
        #print(len(word_list))
        #print(len(nonstop_word_list))
    
        #PART E
        word_counts = collections.Counter(word_list)
        
        nonstop_word_counts = collections.Counter(nonstop_word_list)
        
        save_regular_name=folder+'/'+p.stem+'_'+'regular_saved_dictionary.pkl'
        save_nonstop_name=folder+'/'+p.stem+'_'+'nonstop_saved_dictionary.pkl'
        with open(save_regular_name, 'wb') as f1:
            pickle.dump(word_counts, f1)
        with open(save_nonstop_name, 'wb') as f2:
            pickle.dump(nonstop_word_counts, f2)
        #print(list(word_counts.keys())[:3])
        #print(list(word_counts.values())[:3])
        #print(list(nonstop_word_counts.keys())[:3])
        #print(list(nonstop_word_counts.values())[:3])
        

In [14]:
cd_all_regular=[]
fitz_all_regular=[]
mary_all_regular=[]
myth_all_regular=[]
horror_all_regular=[]
crime_all_regular=[]

cd_all_nonstop=[]
fitz_all_nonstop=[]
mary_all_nonstop=[]
myth_all_nonstop=[]
horror_all_nonstop=[]
crime_all_nonstop=[]

In [33]:
#PART F
folders=["merged_authors"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        number=p.stem.split('-')[1]
        source=p.stem.split('-')[0]
        #stop_type=p.stem.split('_')[1].split('_')[0]
        #print(source+' '+number+' '+stop_type)
        
        save_regular_name=folder+'/'+p.stem+'_'+'regular_saved_dictionary.pkl'
        save_nonstop_name=folder+'/'+p.stem+'_'+'nonstop_saved_dictionary.pkl'
        with open(save_regular_name, 'rb') as f1:
            regular_dict = pickle.load(f1)
        with open(save_nonstop_name, 'rb') as f2:
            nonstop_dict = pickle.load(f2)
        plot_zipf(source,number,regular_dict,'with stopwords')
        plot_zipf(source,number,nonstop_dict,'without stopwords')


        

In [29]:
cd_all_regular=[]
fitz_all_regular=[]
mary_all_regular=[]
myth_all_regular=[]
horror_all_regular=[]
crime_all_regular=[]

cd_all_nonstop=[]
fitz_all_nonstop=[]
mary_all_nonstop=[]
myth_all_nonstop=[]
horror_all_nonstop=[]
crime_all_nonstop=[]
folders=["different_authors"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        number=p.stem.split('-')[1]
        source=p.stem.split('-')[0]
        save_regular_name=folder+'/'+p.stem+'_'+'regular_saved_dictionary.pkl'
        save_nonstop_name=folder+'/'+p.stem+'_'+'nonstop_saved_dictionary.pkl'
        with open(save_regular_name, 'rb') as f1:
            regular_dict = pickle.load(f1)
        with open(save_nonstop_name, 'rb') as f2:
            nonstop_dict = pickle.load(f2)
        #print(int(number))
        if source=='cd':
            cd_all_regular.append(regular_dict)
            cd_all_nonstop.append(nonstop_dict)
        elif source=='fitz':
            fitz_all_regular.append(regular_dict)
            fitz_all_nonstop.append(nonstop_dict)
        elif source=='mary':
            mary_all_regular.append(regular_dict)
            mary_all_nonstop.append(nonstop_dict)


           


In [34]:
plot_zipf_different('cd',cd_all_regular,'with stopwords')
plot_zipf_different('cd',cd_all_nonstop,'without stopwords')
plot_zipf_different('fitz',fitz_all_regular,'with stopwords')
plot_zipf_different('fitz',fitz_all_nonstop,'without stopwords')
plot_zipf_different('mary',mary_all_regular,'with stopwords')
plot_zipf_different('mary',mary_all_nonstop,'without stopwords')


In [44]:
#PART G
stepsize=5000
folders=["merged_authors"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        #print(p)
        number=p.stem.split('-')[1]
        source=p.stem.split('-')[0]
        read_text = open(p, 'rb').read().decode()
        text = read_text.split()
        npunc = [word.strip(string.punctuation) for word in text]
        word_list = [word.lower() for word in npunc if word]
        nonstop_word_list=stopword_removal(word_list)
        #print(range(math.floor(len(word_list)/step_size)))
        #print(range(math.floor(len(nonstop_word_list)/step_size)))
        regular_token_size=[]
        regular_word_type=[]
        nonstop_token_size=[]
        nonstop_word_type=[]
        for i in range(math.floor(len(word_list)/stepsize)):
            [tokens,types]=count_unique(word_list[0:(i+1)*stepsize])
            #print(tokens,types)
            regular_token_size.append(tokens)
            regular_word_type.append(types)
        for i in range(math.floor(len(nonstop_word_list)/stepsize)):
            [tokens,types]=count_unique(nonstop_word_list[0:(i+1)*stepsize])
            nonstop_token_size.append(tokens)
            nonstop_word_type.append(types)
        #print(len(regular_token_size),len(regular_word_type))
        #print(len(nonstop_token_size),len(nonstop_word_type))
        plot_unique(source,regular_token_size,regular_word_type,nonstop_token_size,nonstop_word_type)
        #print(len(word_list))
        #print(len(nonstop_word_list))


In [47]:
#PART H and I
cd_token_regular=[]
fitz_token_regular=[]
mary_token_regular=[]
cd_type_regular=[]
fitz_type_regular=[]
mary_type_regular=[]

cd_token_nonstop=[]
fitz_token_nonstop=[]
mary_token_nonstop=[]
cd_type_nonstop=[]
fitz_type_nonstop=[]
mary_type_nonstop=[]
stepsize=3000
folders=["different_authors"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        number=p.stem.split('-')[1]
        source=p.stem.split('-')[0]
        #print(p)
        read_text = open(p, 'rb').read().decode()
        text = read_text.split()
        npunc = [word.strip(string.punctuation) for word in text]
        word_list = [word.lower() for word in npunc if word]
        nonstop_word_list=stopword_removal(word_list)
        #print(range(math.floor(len(word_list)/step_size)))
        #print(range(math.floor(len(nonstop_word_list)/step_size)))
        regular_token_size=[]
        regular_word_type=[]
        nonstop_token_size=[]
        nonstop_word_type=[]
        for i in range(math.floor(len(word_list)/stepsize)):
            [tokens,types]=count_unique(word_list[0:(i+1)*stepsize])
            regular_token_size.append(tokens)
            regular_word_type.append(types)
        for i in range(math.floor(len(nonstop_word_list)/stepsize)):
            [tokens,types]=count_unique(nonstop_word_list[0:(i+1)*stepsize])
            nonstop_token_size.append(tokens)
            nonstop_word_type.append(types)
        if source=='cd':
            cd_token_regular.append(regular_token_size)
            cd_type_regular.append(regular_word_type)
            cd_token_nonstop.append(nonstop_token_size)
            cd_type_nonstop.append(nonstop_word_type)
        elif source=='fitz':
            fitz_token_regular.append(regular_token_size)
            fitz_type_regular.append(regular_word_type)
            fitz_token_nonstop.append(nonstop_token_size)
            fitz_type_nonstop.append(nonstop_word_type)
        elif source=='mary':
            mary_token_regular.append(regular_token_size)
            mary_type_regular.append(regular_word_type)
            mary_token_nonstop.append(nonstop_token_size)
            mary_type_nonstop.append(nonstop_word_type)

In [1]:
#plot_unique_different('cd',cd_token_regular,cd_type_regular,cd_token_nonstop,cd_type_nonstop)
#plot_unique_different('fitz',fitz_token_regular,fitz_type_regular,fitz_token_nonstop,fitz_type_nonstop)
#plot_unique_different('mary',mary_token_regular,mary_type_regular,mary_token_nonstop,mary_type_nonstop)


In [ ]:
all_token_regular=[]
all_type_regular=[]
all_token_nonstop=[]
all_type_nonstop=[]
all_token_regular.append(cd_token_regular)
all_token_regular.append(fitz_token_regular)
all_token_regular.append(mary_token_regular)
all_type_regular.append(cd_type_regular)
all_type_regular.append(fitz_type_regular)
all_type_regular.append(mary_type_regular)
all_token_nonstop.append(cd_token_nonstop)
all_token_nonstop.append(fitz_token_nonstop)
all_token_nonstop.append(mary_token_nonstop)
all_type_nonstop.append(cd_type_nonstop)
all_type_nonstop.append(fitz_type_nonstop)
all_type_nonstop.append(mary_type_nonstop)
#plot_all_together(all_token_regular,all_type_regular,all_token_nonstop,all_type_nonstop)


In [ ]:
plot_all_together(all_token_regular,all_type_regular,all_token_nonstop,all_type_nonstop)

In [56]:
#PART J
cd_token_regular=[]
fitz_token_regular=[]
mary_token_regular=[]
cd_type_regular=[]
fitz_type_regular=[]
mary_type_regular=[]

cd_token_nonstop=[]
fitz_token_nonstop=[]
mary_token_nonstop=[]
cd_type_nonstop=[]
fitz_type_nonstop=[]
mary_type_nonstop=[]
stepsize=3000
folders=["different_genres"]
for folder in folders:
    paths = list(pathlib.Path(folder).glob("*.txt"))
    for p in paths:
        number=p.stem.split('-')[1]
        source=p.stem.split('-')[0]
        #print(p)
        read_text = open(p, 'rb').read().decode()
        text = read_text.split()
        npunc = [word.strip(string.punctuation) for word in text]
        word_list = [word.lower() for word in npunc if word]
        nonstop_word_list=stopword_removal(word_list)
        #print(range(math.floor(len(word_list)/step_size)))
        #print(range(math.floor(len(nonstop_word_list)/step_size)))
        regular_token_size=[]
        regular_word_type=[]
        nonstop_token_size=[]
        nonstop_word_type=[]
        for i in range(math.floor(len(word_list)/stepsize)):
            [tokens,types]=count_unique(word_list[0:(i+1)*stepsize])
            regular_token_size.append(tokens)
            regular_word_type.append(types)
        for i in range(math.floor(len(nonstop_word_list)/stepsize)):
            [tokens,types]=count_unique(nonstop_word_list[0:(i+1)*stepsize])
            nonstop_token_size.append(tokens)
            nonstop_word_type.append(types)
        if source=='crime':
            cd_token_regular.append(regular_token_size)
            cd_type_regular.append(regular_word_type)
            cd_token_nonstop.append(nonstop_token_size)
            cd_type_nonstop.append(nonstop_word_type)
        elif source=='horror':
            fitz_token_regular.append(regular_token_size)
            fitz_type_regular.append(regular_word_type)
            fitz_token_nonstop.append(nonstop_token_size)
            fitz_type_nonstop.append(nonstop_word_type)
        elif source=='myth':
            mary_token_regular.append(regular_token_size)
            mary_type_regular.append(regular_word_type)
            mary_token_nonstop.append(nonstop_token_size)
            mary_type_nonstop.append(nonstop_word_type)

In [15]:
#plot_unique_different('crime',cd_token_regular,cd_type_regular,cd_token_nonstop,cd_type_nonstop)
#plot_unique_different('horror',fitz_token_regular,fitz_type_regular,fitz_token_nonstop,fitz_type_nonstop)
#plot_unique_different('myth',mary_token_regular,mary_type_regular,mary_token_nonstop,mary_type_nonstop)

In [ ]:
all_token_regular=[]
all_type_regular=[]
all_token_nonstop=[]
all_type_nonstop=[]
all_token_regular.append(cd_token_regular)
all_token_regular.append(fitz_token_regular)
all_token_regular.append(mary_token_regular)
all_type_regular.append(cd_type_regular)
all_type_regular.append(fitz_type_regular)
all_type_regular.append(mary_type_regular)
all_token_nonstop.append(cd_token_nonstop)
all_token_nonstop.append(fitz_token_nonstop)
all_token_nonstop.append(mary_token_nonstop)
all_type_nonstop.append(cd_type_nonstop)
all_type_nonstop.append(fitz_type_nonstop)
all_type_nonstop.append(mary_type_nonstop)


In [ ]:
plot_all_together(all_token_regular,all_type_regular,all_token_nonstop,all_type_nonstop)

In [17]:
#PART M
#random generated corpus
from string import ascii_lowercase
from random import choice
random_corpus=[]
stepsize=3000
#print(letters[26])
lengths=[1,3,5,7,9,11,13,15,17,19,21,25]
for i in range(20*stepsize):
    word_length=choice(lengths)
    random_word=''.join([choice(ascii_lowercase) for _ in range(word_length)])
    random_corpus.append(random_word)
    
#print(len(random_corpus))

regular_token_size=[]
regular_word_type=[]
for i in range(math.floor(len(random_corpus)/stepsize)):
    [tokens,types]=count_unique(random_corpus[0:(i+1)*stepsize])
    regular_token_size.append(tokens)
    regular_word_type.append(types)
plot_unique_just_regular(regular_token_size,regular_word_type)
